In [4]:
import faiss
import numpy as np

d = 64  
nb = 1000 
np.random.seed(123)
db_vectors = np.random.random((nb, d)).astype('float32')
query = np.random.random((1, d)).astype('float32')

index = faiss.IndexFlatL2(d)  
index.add(db_vectors)  
D, I = index.search(query, k=5)  
print("最近邻索引:", I)
print("距离:", D)

最近邻索引: [[482 994 944 913 338]]
距离: [[5.7352233 5.9523897 5.9794054 6.031189  6.267953 ]]


In [5]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')  

sentences = ["this is a sample sentence", "another sentence", "totally different sentence"]

embeddings = model.encode(sentences)

print("shape of embedded vector:", embeddings.shape) 

from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(embeddings)
print("similarity:\n", similarity)

shape of embedded vector: (3, 384)
similarity:
 [[0.99999976 0.6290454  0.45318767]
 [0.6290454  0.99999994 0.6113744 ]
 [0.45318767 0.6113744  0.99999976]]


In [6]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import pickle
import json
import os.path

class VectorDatabase:
    def __init__(self, model_name='all-MiniLM-L6-v2', index_path='faiss_index.bin', metadata_path='metadata.json', max_seq_length=512):
        self.model = SentenceTransformer(model_name)
        self.model.max_seq_length = max_seq_length   
        self.dimension = self.model.get_sentence_embedding_dimension() 
        self.index = faiss.IndexFlatL2(self.dimension)  
        self.index_path = index_path
        self.metadata_path = metadata_path
        self.metadata = []  
        self.next_id = 0
        self.load()

    def add(self, texts, metadata=None):
        embeddings = self.model.encode(
            texts,
            convert_to_numpy=True,
            normalize_embeddings=True,
            show_progress_bar=True,
            batch_size=16
        ).astype('float32')

        self.index.add(embeddings)

        if metadata is None:
            metadata = [{} for _ in texts]
        for i, text in enumerate(texts):
            self.metadata.append({
                'id': self.next_id,
                'text': text,
                'extra': metadata[i]
            })
            self.next_id += 1

        self.save()

    def search(self, query_text, k=5):
        query_embedding = self.model.encode(
            [query_text],
            convert_to_numpy=True,
            normalize_embeddings=True,
            show_progress_bar=True,
            batch_size=1
        ).astype('float32')

        distances, indices = self.index.search(query_embedding, k)

        results = []
        for idx, dist in zip(indices[0], distances[0]):
            if idx != -1 and idx < len(self.metadata):
                results.append({
                    'distance': float(dist),
                    'metadata': self.metadata[idx]
                })

        return results

    def delete(self, id):
        if id in [m['id'] for m in self.metadata]:
            self.metadata = [m for m in self.metadata if m['id'] != id]

            embeddings = self.model.encode(
                [m['text'] for m in self.metadata],
                convert_to_numpy=True,
                normalize_embeddings=True,
                show_progress_bar=True,
                batch_size=16
            ).astype('float32')
            self.index = faiss.IndexFlatL2(self.dimension)
            if len(embeddings) > 0:
                self.index.add(embeddings)

            self.save()

    def save(self):
        faiss.write_index(self.index, self.index_path)
        with open(self.metadata_path, 'w', encoding='utf-8') as f:
            json.dump(self.metadata, f, ensure_ascii=False, indent=2)

    def load(self):
        if os.path.exists(self.index_path):
            self.index = faiss.read_index(self.index_path)
        if os.path.exists(self.metadata_path):
            try:
                with open(self.metadata_path, 'r', encoding='utf-8') as f:
                    content = f.read().strip()
                    if content:
                        self.metadata = json.loads(content)
                        self.next_id = max([m['id'] for m in self.metadata], default=-1) + 1
                    else:
                        self.metadata = []
                        self.next_id = 0
            except Exception as e:
                print(f"加载元数据文件出错: {e}, 已重置为空.")
                self.metadata = []
                self.next_id = 0

# 使用示例
if __name__ == "__main__":
    # 初始化数据库
    db = VectorDatabase(
        model_name='all-MiniLM-L6-v2',
        index_path='vector_db_index.bin',
        metadata_path='vector_db_metadata.json',
        max_seq_length=512
    )

    # 添加大段中文文本
    texts = [
        "人工智能（AI）是计算机科学的一个分支，旨在创建能够执行需要人类智能的任务的系统，例如学习、问题解决、决策和感知。近年来，AI 技术在自然语言处理、计算机视觉和机器人领域取得了显著进展。",
        "机器学习是人工智能的核心子领域，它使计算机能够通过数据和经验进行学习，而无需明确的编程。监督学习、无监督学习和强化学习是机器学习的主要类型。",
        "深度学习是一种基于人工神经网络的机器学习技术，广泛应用于图像识别、语音处理和自然语言处理。深度学习的成功得益于大数据、强大的计算能力和优化的算法。"
    ]
    metadata = [
        {'category': 'AI Overview'},
        {'category': 'Machine Learning'},
        {'category': 'Deep Learning'}
    ]
    db.add(texts, metadata)

    # 查询
    query = "人工智能技术如何影响自然语言处理？"
    results = db.search(query, k=2)
    print("查询结果:")
    for result in results:
        print(f"距离: {result['distance']:.4f}, 元数据: {result['metadata']}")

    # 删除（示例）
    db.delete(1)  # 删除 ID 为 1 的记录

    # 再次查询
    results = db.search(query, k=2)
    print("\n删除后查询结果:")
    for result in results:
        print(f"距离: {result['distance']:.4f}, 元数据: {result['metadata']}")

Batches: 100%|██████████| 1/1 [00:00<00:00, 175.77it/s]



查询结果:
距离: 0.4142, 元数据: {'id': 0, 'text': '人工智能（AI）是计算机科学的一个分支，旨在创建能够执行需要人类智能的任务的系统，例如学习、问题解决、决策和感知。近年来，AI 技术在自然语言处理、计算机视觉和机器人领域取得了显著进展。', 'extra': {'category': 'AI Overview'}}
距离: 0.4142, 元数据: {'id': 3, 'text': '人工智能（AI）是计算机科学的一个分支，旨在创建能够执行需要人类智能的任务的系统，例如学习、问题解决、决策和感知。近年来，AI 技术在自然语言处理、计算机视觉和机器人领域取得了显著进展。', 'extra': {'category': 'AI Overview'}}


Batches: 100%|██████████| 1/1 [00:00<00:00, 145.04it/s]


删除后查询结果:
距离: 0.4142, 元数据: {'id': 0, 'text': '人工智能（AI）是计算机科学的一个分支，旨在创建能够执行需要人类智能的任务的系统，例如学习、问题解决、决策和感知。近年来，AI 技术在自然语言处理、计算机视觉和机器人领域取得了显著进展。', 'extra': {'category': 'AI Overview'}}
距离: 0.4142, 元数据: {'id': 3, 'text': '人工智能（AI）是计算机科学的一个分支，旨在创建能够执行需要人类智能的任务的系统，例如学习、问题解决、决策和感知。近年来，AI 技术在自然语言处理、计算机视觉和机器人领域取得了显著进展。', 'extra': {'category': 'AI Overview'}}
